# Défi Grosses Data 2018

## Début Commun pour tous

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)

Importe les données concaténées:

In [ ]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

In [ ]:
df=Annex.get_data_raw()
N_withNA=df.shape[0]

Importe les données séparées par type:

In [ ]:
meteo_quant, meteo_qual, meteo_date, meteo_y=Annex.get_data_tidied()

Remplace les variables qualitatives par leur indicatrices. 

**Attention, ici seuls "mois" et "insee" sont considérés qualitatives!** (what about the wind? what about the "ech"?)

In [ ]:
df=Annex.convert_month_to_int(df)

In [ ]:
df_dummies=pd.get_dummies(df[['insee','ddH10_rose4']])
df_full_qtt=pd.concat([df,df_dummies],axis=1)
df_full_qtt=df_full_qtt.drop(['insee','ddH10_rose4'],axis=1)

Sépare les échantillons d'apprentissage et de test

In [ ]:
#df_full_qtt.isnull().values.any()
df_full_qtt.isnull().sum()

In [ ]:
df_clean=df_full_qtt.dropna(axis=0)
N_withoutNA=df_clean.shape[0]
print("Nous avons éliminé %d données soit %0.2f %s"%(N_withNA-N_withoutNA,(N_withNA-N_withoutNA)/N_withNA*100,'%'))

In [ ]:
from sklearn.model_selection import train_test_split
Y=df_clean['tH2_obs']
X=df_clean
X=X.drop(['tH2_obs'],axis=1) ## !!! Date?
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=11)
X_train.shape, X_test.shape

## Maintenant, faites votre vie!

In [ ]:
from sklearn.preprocessing import StandardScaler  
# L'algorithme ds réseaux de neurones nécessite éventuellement une normalisation 
# des variables explicatives avec les commandes ci-dessous
date_train=X_train['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
date_test=X_test['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
X_train=X_train.drop(['date'],axis=1)
X_test=X_test.drop(['date'],axis=1)

scaler = StandardScaler()  
scaler.fit(X_train)  
Xnet_train = scaler.transform(X_train)  
# Meme transformation sur le test
Xnet_test = scaler.transform(X_test)
#date_train=np.reshape(date_train,(len(date_train),1))
#date_test=np.reshape(date_test,(len(date_test),1))
#Xnet_train.shape,date_train.shape,type(X_train),type(date_train)

#Xnet_train=np.concatenate((Xnet_train,date_train),axis=1)
#Xnet_test=np.concatenate((Xnet_test,date_test),axis=1)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
param_grid=[{"hidden_layer_sizes":list([(100,)])}]#(50,),(60,),(70,),(80,),,(120,)
nnet= GridSearchCV(MLPRegressor(max_iter=500),param_grid,cv=10,n_jobs=-1)
nnetOpt=nnet.fit(X_train, Y_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - nnetOpt.best_score_,nnetOpt.best_params_))

In [ ]:
from sklearn.metrics import mean_squared_error
import pyGPs # Library for Gaussian Processes
import sys
sys.path.append("./libs")
import presentation as pr
## Creation Model pyGPs
model=pyGPs.GPR()
m=pyGPs.mean.Const(0.4)
k=pyGPs.cov.RBF(log_ell=-2.5,log_sigma=0)
model.setPrior(mean=m,kernel=k)

## Momentum
mom=[]
momentum=0.7
start_m=0.5
end_m=0.99

## Learning Rate
l=[]
learning_rate=0.5
start_lr=-4#1e-6
end_lr=-2#5e-4

## Nb Nodes
nn=[]
nb_nodes=0.5
start_nn=20
end_nn=200

sz_space=100

v=np.reshape(np.linspace(0,1,sz_space),(sz_space,1))
v_mom=np.reshape(np.linspace(0,1,sz_space),(sz_space,1))
X1,X2=np.meshgrid(v,v_mom)
V=np.concatenate([np.reshape(X1,(sz_space**2,1)),np.reshape(X2,(sz_space**2,1))],axis=1)
v=np.reshape(v,(len(v),))

nb_div=0
best_so_far=np.Inf
RMSE=[]
for i in range(10):
    l=np.append(l,learning_rate)
    mom=np.append(mom,momentum)

    HP=np.concatenate([np.reshape(l,(len(l),1)),np.reshape(mom,(len(mom),1))],axis=1)
    if i>1 and ((np.sum(np.abs(HP[-1,:]-HP[-2,:]))<1e-5 and np.sum(np.abs(HP[-1,:]))>1e-7) or ys2[ind_min]<1.4e-2):
        print('MINIMA MIGHT BE FOUND, quitting iteration',i)
        print(np.sum(np.abs(HP[-1,:]-HP[-2,:])),HP[-1,:],HP[-2,:])
        print('ys2[ind_min]',ys2[ind_min])
        break
    pr.big_iter(i+1,'Learning rate:%0.2e, Momentum:%0.2f, Best so far:%0.4f'% (10**(learning_rate*(end_lr-start_lr)+start_lr),momentum*(end_m-start_m)+start_m,best_so_far))

    ## Evaluation
    alpha=10**(learning_rate*(end_lr-start_lr)+start_lr)
    beta=momentum*(end_m-start_m)+start_m
    mlp=MLPRegressor(max_iter=500,hidden_layer_sizes =(80,),learning_rate='constant',learning_rate_init=alpha,momentum=beta)
    nnetOpt=mlp.fit(Xnet_train, Y_train)
    Y_pred = nnetOpt.predict(Xnet_test)
    score=mean_squared_error(Y_test,Y_pred)
    RMSE=np.append(RMSE,score)
    if RMSE[-1]<best_so_far:
        best_so_far=RMSE[-1]
        best_net=mlp
        #best_ind=
    
    ## Gaussian Processes
    print(HP,RMSE)
    model.getPosterior(HP,RMSE)
    # model.optimize(l,Y)
    ym,ys2,fm,fs2,lp=model.predict(V)
    # print(ym.shape,V.shape)

    ## Updates
    ind_min=np.argmin(ym-ys2)
    learning_rate=V[ind_min,0]
    momentum=V[ind_min,1]
    print('RMSE',score,', S2:%10.2e'%ys2[ind_min])
    ## Plots
    

## Final Outputs
pr.line()
print('Last learning rate:',10**(l[-1]*(end_lr-start_lr)+start_lr),'\nMomentum:',mom[-1]*(end_m-start_m)+start_m)
print('Last Result:',score)

pr.line()
#print('Best result:\n',best_ind,best_net,best_so_far)
HP=HP[:-1,:]
print(HP)
#plots('log')




In [ ]:
help(MLPRegressor)

In [ ]:
mlp=MLPRegressor(max_iter=500,hidden_layer_sizes =(50,))
nnetOpt=mlp.fit(Xnet_train, Y_train)

In [ ]:
# Estimation de l'erreur de prévision sur le test
1-nnetOpt.score(X_test,Y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(Xnet_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

In [ ]:
X_train.dtypes

In [ ]:
Y_train=Y_train.reshape((len(Y_train),1))
Y_test=Y_test.reshape((len(Y_test),1))
import sys
sys.path.append("./libs")
import NNet
import imp
imp.reload(NNet)
n_output=[Y_train.shape[1]]
n_hid=[7]
n_input=[X_train.shape[1]]

type_layer=['logistic','linear']
threshold=1e-4
max_iter=1000
mini_batch_coef=0.1
do_early_stopping=False

min_training_loss=np.Inf
min_val_loss=np.Inf
min_test_loss=np.Inf

i=0
for alpha in [0.35]:#[0.002,0.01,0.05,0.2,1,5]:
    for mom in [0.9]:
        for wd in [0]:#[0.001,0,1,0.1,10,0.0001]:
            for n_hid in [[15]]:#[30,20],,[40,30],[10,15],[20,20]
                i=i+1
                sizes=np.concatenate([n_input,n_hid,n_output])
                print('==================== Test n°',i,' alpha=',alpha,' mom=',mom,' wd=',wd,' struct=',sizes)
                [model,losses]=NNet.build_model(X_train.T, Y_train.T, wd, sizes, type_layer, 'least_square',
                                                   max_iter, alpha, mom,
                                                   do_early_stopping, mini_batch_coef, threshold,
                                                   print_info=True,train_test_split=False,
                                                   X_test=X_test.T,y_test=Y_test.T)
                if losses[0]<min_training_loss:
                    min_training_loss=losses[0]
                    min_alpha=alpha
                    momentum=mom
                    wd_t=wd
                    n_hid_t=n_hid
                if losses[1]<min_val_loss:
                    min_val_loss=losses[1]
                    min_alpha_val=alpha
                    momentum_val=mom
                    wd_v=wd
                    n_hid_v=n_hid
                if losses[2]<min_test_loss:
                    min_test_loss=losses[2]
                    min_alpha_test=alpha
                    momentum_test=mom
                    wd_test=wd
                    n_hid_test=n_hid
                    # best_class_perf=class_perf[2]

print(i, ' configurations have been tested' )

print('==== Training test')
print('Min alpha: ',min_alpha)
print('Momentum: ',momentum)
print('Min val loss: ',min_training_loss)
print('WD: ',wd_t)
print('n_hid: ',n_hid_t)

print('==== Validation test')
print('Min alpha: ',min_alpha_val)
print('Momentum: ',momentum_val)
print('Min val loss: ',min_val_loss)
print('WD: ',wd_v)
print('n_hid: ',n_hid_v)

print('==== Generalisation test')
print('Min alpha: ',min_alpha_test)
print('Momentum: ',momentum_test)
print('Min val loss: ',min_test_loss)
print('WD: ',wd_test)
print('n_hid: ',n_hid_test)
# print('Classification performance: ',best_class_perf)

print('\nElapsed time:',time.time()-start)

In [ ]:
df_test=Annex.load_test_set()
np.sum(is.na(df_test))

In [ ]:
imp.reload(Annex)
df_TEST=Annex.load_test_set()
#df_TEST.shape, df_TEST.info()
df_TEST=Annex.convert_month_to_int(df_TEST)
df_dummies=pd.get_dummies(df_TEST[['insee']])
df_TEST_full_qtt=pd.concat([df_TEST,df_dummies],axis=1)
df_TEST_full_qtt.flir1SOL0=df_TEST_full_qtt.flir1SOL0.fillna(0)
df_TEST_full_qtt.fllat1SOL0=df_TEST_full_qtt.fllat1SOL0.fillna(0)
df_TEST_full_qtt.flsen1SOL0=df_TEST_full_qtt.flsen1SOL0.fillna(0)
df_TEST_full_qtt.flvis1SOL0=df_TEST_full_qtt.flvis1SOL0.fillna(0)
df_TEST_full_qtt.rr1SOL0=df_TEST_full_qtt.rr1SOL0.fillna(0)
df_TEST_full_qtt=df_TEST_full_qtt.drop(['insee','date'],axis=1)
X_TEST = scaler.transform(df_TEST_full_qtt)  
Y_PRED = nnetOpt.predict(X_TEST)

## Submission

In [ ]:
Annex.generate_submission_file('submission_17nov2017_10h30.csv', Y_PRED)